In [1]:
#Opening a New Indian Restaurant in Toronto, canada

#Build a dataframe of neighborhoods in Toronto, Canada by web scraping the data from Wikipedia page
#Get the geographical coordinates of the neighborhoods
#Obtain the venue data for the neighborhoods from Foursquare API
#Explore and cluster the neighborhoods
#Select the best cluster to open a new Indian restaurant


!pip install folium
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium


     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [2]:
# Getting the list of neighborhoods in Toronto, Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')

Page download successful


In [3]:
# Converting html page into dataframe
toronto = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#Drop the the rows on which the Borough is empty
toronto.dropna(subset=['Borough'], inplace=True)

#Group by Postcode / Borough
postcodes = toronto.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)])
postcodes.reset_index(inplace=True)
postcodes.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
print('The shape of the dataset is:',postcodes.shape)

The shape of the dataset is: (103, 3)


In [6]:
#Export to .CSV
postcodes.to_csv('Toronto_Postcodes.csv')

In [7]:
#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
coordinates = pd.read_csv(url_csv)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#use the previously cleaned data
neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
# Make sure both dataframes have the same 
coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
neighborhoods.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)


In [10]:
# Merge both datasets
neighborhoods_coordinates = pd.merge(neighborhoods, coordinates, on='PostalCode')
neighborhoods_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
#Export to .CSV
neighborhoods_coordinates.to_csv('Toronto-2.csv')

In [12]:
# Read csv file from above
toronto_df= pd.read_csv('Toronto-2.csv', index_col=0)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
#count Bourough and Neighborhood
toronto_df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [14]:
#Create list with the Boroughs (to be used later)
boroughs = toronto['Borough'].unique().tolist()

In [15]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
latitude_toronto = toronto_df['Latitude'].mean()
longitude_toronto = toronto_df['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(latitude_toronto, longitude_toronto))

The geographical coordinates of Toronto are 43.70460773398059, -79.39715291165048


In [16]:

borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))
    
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], 
                                           toronto_df['Longitude'],
                                           toronto_df['Borough'], 
                                           toronto_df['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto    

In [17]:
# Getting Venues Data using Foursquare

CLIENT_ID = 'VAH25GVAB0FYVVECSXU2TTQ1YPAO4MS1LU1ZMWHCBREJ2J1U' 
CLIENT_SECRET = 'U11TK4UIPXXCDFFYBQW4R042T4NMIWASVZJJPTHDQCYLUJYP'
VERSION = '20200701' 
LIMIT = 100 
radius = 500 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                latitudes=toronto_df['Latitude'],
                                longitudes=toronto_df['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [19]:
#Check size of resulting dataframe
toronto_venues.shape

(2132, 7)

In [20]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [21]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15


In [22]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [23]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Fast Food Restaurant', 'Bar', 'Bank', 'Electronics Store',
       'Restaurant', 'Mexican Restaurant', 'Rental Car Location',
       'Medical Center', 'Intersection', 'Breakfast Spot', 'Coffee Shop',
       'Korean Restaurant', 'Pharmacy', 'Hakka Restaurant',
       'Caribbean Restaurant', 'Thai Restaurant', 'Athletics & Sports',
       'Bakery', 'Gas Station', 'Fried Chicken Joint', 'Playground',
       'Department Store', 'Chinese Restaurant', 'Hobby Shop',
       'Bus Station', 'Ice Cream Shop', 'Bus Line', 'Metro Station',
       'Park', 'Motel', 'American Restaurant', 'Café',
       'General Entertainment', 'Skating Rink', 'College Stadium',
       'Indian Restaurant', 'Pet Store', 'Vietnamese Restaurant',
       'Light Rail Station', 'Gaming Cafe', 'Sandwich Place',
       'Middle Eastern Restaurant', 'Smoke Shop', 'Auto Garage', 'Lounge',
       'Latin American Restaurant', 'Clothing Store',
       'Italian Restaurant', 'Noodle House', 'Pizza Place',
       'Grocery Store

In [24]:
#Analysing each neighborhood

# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(2132, 271)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(95, 271)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [26]:
len(to_grouped[to_grouped["Indian Restaurant"] > 0])

11

In [27]:
indian = to_grouped[["Neighborhoods","Indian Restaurant"]]
indian.head()

,Neighborhoods,Indian Restaurant
0,Agincourt,0.000000
1,"Alderwood, Long Branch",0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000
3,Bayview Village,0.000000
4,"Bedford Park, Lawrence Manor East",0.043478


In [28]:
# Run k-means to cluster the neighborhoods in Toronto into 3 clusters.

toclusters = 3

to_clustering = indian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = indian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels
0,Agincourt,0.000000,0
1,"Alderwood, Long Branch",0.000000,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0
3,Bayview Village,0.000000,0
4,"Bedford Park, Lawrence Manor East",0.043478,2


In [30]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(2132, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,0.0,0,43.7942,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
0,Agincourt,0.0,0,43.7942,-79.262029,Twilight,43.791999,-79.258584,Lounge
0,Agincourt,0.0,0,43.7942,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant
0,Agincourt,0.0,0,43.7942,-79.262029,Mark's,43.791179,-79.259714,Clothing Store
0,Agincourt,0.0,0,43.7942,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink


In [31]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(2132, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,0.000000,0,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,M Square Coffee Co,43.651218,-79.383555,Coffee Shop
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,John & Sons Oyster House,43.650656,-79.381613,Seafood Restaurant
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Soho House Toronto,43.648734,-79.386541,Speakeasy
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Pizzeria Libretto,43.648334,-79.385111,Pizza Place
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
64,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant


In [32]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [33]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# # Examine clusters to find location

In [34]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,0.0,0,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,M Square Coffee Co,43.651218,-79.383555,Coffee Shop
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,John & Sons Oyster House,43.650656,-79.381613,Seafood Restaurant
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Soho House Toronto,43.648734,-79.386541,Speakeasy
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Pizzeria Libretto,43.648334,-79.385111,Pizza Place
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
64,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant


In [35]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
23,"Dorset Park, Wexford Heights, Scarborough Town...",0.285714,1,43.75741,-79.273304,Kairali,43.754915,-79.276945,Indian Restaurant
23,"Dorset Park, Wexford Heights, Scarborough Town...",0.285714,1,43.75741,-79.273304,Omescape Scarborough,43.754158,-79.276230,Gaming Cafe
23,"Dorset Park, Wexford Heights, Scarborough Town...",0.285714,1,43.75741,-79.273304,Scarborough LRT,43.756465,-79.272194,Light Rail Station
23,"Dorset Park, Wexford Heights, Scarborough Town...",0.285714,1,43.75741,-79.273304,Pho Vietnam,43.757770,-79.278572,Vietnamese Restaurant
23,"Dorset Park, Wexford Heights, Scarborough Town...",0.285714,1,43.75741,-79.273304,Big Al's Pet Supercentre,43.759279,-79.278325,Pet Store
23,"Dorset Park, Wexford Heights, Scarborough Town...",0.285714,1,43.75741,-79.273304,Karaikudi Chettinad South Indian Restaurant,43.756042,-79.276276,Indian Restaurant
23,"Dorset Park, Wexford Heights, Scarborough Town...",0.285714,1,43.75741,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant


In [36]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,The Copper Chimney,43.736195,-79.420271,Indian Restaurant
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Satay on the Road,43.735310,-79.419783,Thai Restaurant
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Tim Hortons,43.735356,-79.419605,Coffee Shop
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Sakura Garden,43.733398,-79.419491,Sushi Restaurant
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Pheasant & Firkin,43.735173,-79.419702,Pub
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Freshii,43.731582,-79.419109,Juice Bar
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Drums N Flats,43.735035,-79.420040,Comfort Food Restaurant
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Darbar Persian Grill,43.735484,-79.420006,Restaurant
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Francobollo,43.734557,-79.419549,Italian Restaurant
4,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Aroma Espresso Bar,43.735975,-79.420391,Café
